In [1]:
%pip install plotly --quiet
%pip install pandas --quiet
%pip install ta --quiet
import ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import json
import datetime
import pandas as pd
import numpy as np
import time
import os
from math import ceil

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Use free ProtonVPN. Download bulk binance data here when needed.
interval = '5m'
symbol = 'ETHUSDT'
limit = 1000
# number of: (months * days * hours * minutes * seconds) * miliseconds
start_time = int((time.time() - 30 * 30 * 24 * 60 * 60) * 1000)
candle_data = []

while True:
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}&startTime={start_time}'
    request = requests.get(url)
    data = request.json()

    if len(data) == 0:  
        break

    candle_data += data
    start_time = data[-1][0] + 1
    time.sleep(0.1)

with open(f'{symbol}.json', 'w') as f:
    json.dump(candle_data, f)

df = pd.DataFrame(candle_data, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base', 'Taker buy quote', 'Ignore'])
df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
df.to_csv(f'{symbol}.csv', index=False)

In [2]:
btc_usdt_df = pd.read_csv('BTCUSDT.csv')
eth_usdt_df = pd.read_csv('ETHUSDT.csv')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

dataframes = [btc_usdt_df, eth_usdt_df]

for df in dataframes:
    df.columns = map(str.lower, df.columns)
    df.drop(['close time', 'volume', 'taker buy base', 'taker buy quote', 'ignore'], axis=1, inplace=True)
    df.rename(columns={'quote asset volume': 'volume_usd', 'number of trades': 'num_trades', 'open time': 'time'}, inplace=True)
    df['time'] = pd.to_datetime(df['time'])
    df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14)
    df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    df['sma50'] = ta.trend.sma_indicator(df['close'], window=50)
    df['sma100'] = ta.trend.sma_indicator(df['close'], window=100)
    df['sma200'] = ta.trend.sma_indicator(df['close'], window=200)
    df[['atr', 'rsi', 'sma50', 'sma100','sma200']] = df[['atr', 'rsi', 'sma50', 'sma100','sma200']].round(2)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

In [3]:
btc_usdt_df = btc_usdt_df.head(40000)
btc_usdt_df

,time,open,high,low,close,volume_usd,num_trades,atr,rsi,sma50,sma100,sma200
0,2020-12-17 06:10:00,22132.26,22133.35,22053.00,22089.95,9.270872e+06,6276,87.82,58.28,21907.20,21660.13,21168.37
1,2020-12-17 06:15:00,22089.95,22136.59,22063.81,22133.99,5.376863e+06,4869,86.74,60.88,21915.80,21669.54,21179.80
2,2020-12-17 06:20:00,22134.00,22230.00,22113.00,22230.00,1.160161e+07,7891,88.90,65.88,21924.50,21678.81,21189.80
3,2020-12-17 06:25:00,22229.99,22271.45,22190.00,22269.14,9.205871e+06,7432,88.37,67.70,21933.54,21688.38,21199.92
4,2020-12-17 06:30:00,22269.15,22400.00,22099.55,22111.57,3.789862e+07,17642,103.52,55.02,21938.49,21696.47,21208.82
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,2021-05-05 18:10:00,57439.10,57439.78,57338.30,57425.80,9.053189e+06,5653,193.83,54.61,57124.04,56305.56,55573.37
39996,2021-05-05 18:15:00,57425.80,57436.09,57256.39,57287.83,9.501391e+06,5914,192.82,47.77,57153.72,56324.85,55586.31
39997,2021-05-05 18:20:00,57287.83,57365.12,57210.36,57274.46,9.861144e+06,5695,190.10,47.15,57177.80,56343.67,55599.20
39998,2021-05-05 18:25:00,57274.46,57400.00,57270.67,57319.05,7.437879e+06,5246,185.76,49.49,57208.83,56363.88,55611.86


In [8]:
########### find_horizontal_resistance_lines ###########

#--> Identifies an "origin" green candle for resistance line starting point. 
#--> Defines the resistance 'line_price' as a weighted mean of the origin candle's high and close prices. 
# (This is superior to HLC/3 because we are only interested in the high wicks for resistance lines). 
#--> The 'waiting_period' and the 'waiting_candles' is a way to separate 'clumped' candle touches to have min # candles between origin point and 2nd touch. 
#--> Checks the following candles for touch points, using % of atr as wiggle room to define an acceptable range for touch points.
#--> Makes sure there are no overlapping lines during line creation.

########### find_horizontal_resistance_lines ###########

In [4]:
def find_horizontal_resistance_lines(df):
    resistance_lines = []
    i = 0
    waiting_period = 10

    # Start a loop over the DataFrame, leaving space for checking a waiting period.
    while i < len(df) - waiting_period: 
        origin_candle = df.iloc[i]

        # Only consider green candles as potential origin points. Could also consider red ones and pick max of 'close', 'open', but isn't as clean.
        if origin_candle['close'] > origin_candle['open']:
            # Define line price as weighted mean of close and high. Set 'second_touch' flag to None.
            line_price = 0.6 * origin_candle['close'] + 0.4 * origin_candle['high'] 
            second_touch = None 

            # Define an acceptable range for the waiting candles. Uses 'atr' of current iterated candle. 
            waiting_candles = df.iloc[i+1:i+waiting_period+1]
            waiting_candles_upper_range = line_price + (0.5 * waiting_candles['atr'])
            waiting_candles_line_price = 0.6 * waiting_candles[['close', 'open']].max(axis=1) + 0.4 * waiting_candles['high']
            
            # Skip to the next 'origin' candle if any waiting candle's close prices exceed the upper limit.
            if any(waiting_candles_line_price > waiting_candles_upper_range):
                i += 1
                continue

            # If all waiting candles are within the range, start looking for the second touch point. Uses 'atr' of current iterated candle.   
            for j in range(i+waiting_period, len(df)):  
                candle = df.iloc[j]

                # Define the upper and lower acceptable touch ranges for the next candle. 
                # 'bottom_limit' is useful for if the price goes way below resistance line, stop the line.
                upper_range = line_price + (0.5 * candle['atr'])
                lower_range = line_price - (0.5 * candle['atr'])
                bottom_limit = line_price - (5 * candle['atr'])

                # 'candle' are the candles when looking for the 2nd touch point.
                # 'bottom' refers to the weighted line price we are going to use when looking for if the 'bottom_limit' gets hit.
                candle_line_price = 0.6 * candle[['close', 'open']].max() + 0.4 * candle['high']
                candle_line_price_bottom = 0.6 * candle[['close', 'open']].min() + 0.4 * candle['low']

                # Check if the candle is a touch point.
                if lower_range <= candle_line_price <= upper_range and second_touch is None:
                    second_touch = candle['time']
                    touch_2_price = candle_line_price
                
                # Check if the candle is outside of the acceptable range or bottom_limit
                if candle_line_price > upper_range or candle_line_price_bottom < bottom_limit:
                    # If there was a second touch, save the line to the list of resistance lines.
                    if second_touch is not None:
                        resistance_lines.append({'initial_point': origin_candle['time'], 'end_of_line': candle['time'], 'touch_2': second_touch, 'touch_2_price': touch_2_price, 'line_price': line_price})
                    second_touch = None  
                    i = j + 1  # Start analyzing from the next candle
                    break
            else:
                i += 1  # If the loop finishes without breaking, check the next candle.
        else:
            i += 1  # If the candle is not green, check the next one.

    return pd.DataFrame(resistance_lines)

resistance_df = find_horizontal_resistance_lines(btc_usdt_df)

In [6]:
time_interval_in_minutes = 5
resistance_df['initial_point'] = pd.to_datetime(resistance_df['initial_point'])
resistance_df['end_of_line'] = pd.to_datetime(resistance_df['end_of_line'])

# Calculates length of horizontal resistance line based on time interval. Returns # of candles as a value.
resistance_df['line_length'] = (resistance_df['end_of_line'] - resistance_df['initial_point']).dt.total_seconds() / (time_interval_in_minutes * 60)
resistance_df['line_length'] = resistance_df['line_length'].astype(int)

In [7]:
resistance_df

,initial_point,end_of_line,touch_2,touch_2_price,line_price,line_length
0,2020-12-17 06:25:00,2020-12-17 07:20:00,2020-12-17 07:15:00,22288.760,22270.064,11
1,2020-12-17 09:25:00,2020-12-17 13:05:00,2020-12-17 12:55:00,23169.594,23162.428,44
2,2020-12-17 13:35:00,2020-12-17 16:05:00,2020-12-17 15:50:00,23270.470,23326.852,30
3,2020-12-17 16:30:00,2020-12-17 18:15:00,2020-12-17 17:20:00,23587.012,23642.482,21
4,2020-12-17 20:50:00,2020-12-17 22:00:00,2020-12-17 21:40:00,22696.168,22721.138,14
...,...,...,...,...,...,...
746,2021-05-04 18:45:00,2021-05-04 20:50:00,2021-05-04 19:55:00,54552.626,54619.570,25
747,2021-05-04 22:55:00,2021-05-05 00:30:00,2021-05-05 00:25:00,54050.680,54103.486,19
748,2021-05-05 01:10:00,2021-05-05 02:10:00,2021-05-05 02:05:00,54928.126,55003.754,12
749,2021-05-05 02:15:00,2021-05-05 06:15:00,2021-05-05 04:30:00,55280.944,55331.866,48


In [7]:
########### find_touch_points ###########

#--> After the initial two points of a resistance line are established, this function finds further touch points.
#--> Utilizes a 'waiting_period' to ensure some distance between subsequent touch points, preventing clumping.
#--> Defines the acceptable range for touch points using a % of atr as a threshold. Checks the df for touch points within this range.
#--> If a touch point is found, the function records it's price and timestamp, and starts the search for the next touch point after the waiting period.
#--> If the price moves outside of the acceptable range before another touch point is found, or if the 'bottom_limit' is crossed, the line ends. 
#--> Iterates over the resistance_df until all lines have been evaluated for further touch points.

########### find_touch_points ###########

In [8]:
def find_touch_points(df, resistance_df):

    # Set waiting period between touches, initialize counter for max touch points, initialize list of column names for touch points and prices.
    waiting_period = 10
    max_touch_points = 0
    touch_point_cols = []
    touch_price_cols = []
    
    # Iterate over the resistance lines DataFrame.
    for idx, row in resistance_df.iterrows():
        # Define the start index (just after the second touch point), end index and the resistance line price.
        second_touch_point = row['touch_2']
        price = row['line_price']
        start_idx = df.loc[df['time'] == second_touch_point].index[0] + waiting_period
        end_idx = df.loc[df['time'] == row['end_of_line']].index[0]

        # Initialize dict for touch points and prices.
        touch_points = {}
        touch_prices = {}

        # Loop over the df from the start index to the end index.
        while start_idx < end_idx:
            slice_df = df.iloc[start_idx:end_idx+1]  

            # Initialize a flag for found touch points.
            found = False

            # Loop over each candle within the sliced DataFrame.
            for j, candle in slice_df.iterrows():
                # Define the upper and lower acceptable ranges for touch points.
                # 'bottom_limit' is useful for if the price goes way below resistance line, stop the line.
                upper_range = price + (0.5 * candle['atr'])
                lower_range = price - (0.5 * candle['atr'])
                bottom_limit = price - (5 * candle['atr'])

                candle_line_price = 0.6 * candle[['close', 'open']].max() + 0.4 * candle['high']
                candle_line_price_bottom = 0.6 * candle[['close', 'open']].min() + 0.4 * candle['low']

                # Check if the candle is within the acceptable range for a touch point.
                if lower_range <= candle_line_price <= upper_range:
                    touch_points[len(touch_points)+3] = candle['time']
                    touch_prices[len(touch_prices)+3] = candle_line_price
                    # Start the next search after the waiting period.
                    start_idx = j + waiting_period
                    found = True
                    break
                # Check if the candle is outside of the acceptable range or bottom_limit.
                if candle_line_price > upper_range or candle_line_price_bottom < bottom_limit:
                    break
            # If no touch point was found, break the loop.                
            if not found:
                break

        # Update the maximum number of touch points.
        max_touch_points = max(max_touch_points, len(touch_points))

        # Loops over found touch points and adds them to resistance_df.
        for i, touch_point in touch_points.items():
            # Define column names for touch points and their prices.
            col_name = f'touch_point_{i}'
            price_col_name = f'touch_point_{i}_price'

            # Add new columns (timestamp and price) to the DataFrame if they don't exist.
            if col_name not in touch_point_cols:
                touch_point_cols.append(col_name)
                resistance_df[col_name] = None
            if price_col_name not in touch_price_cols:
                touch_price_cols.append(price_col_name)
                resistance_df[price_col_name] = None

            # Add the touch points (timestamps) and their prices to the df.
            resistance_df.loc[idx, col_name] = touch_point
            resistance_df.loc[idx, price_col_name] = touch_prices[i]
        # Record the total number of touch points for the line in the df.
        resistance_df.loc[idx, 'num_touch_points'] = len(touch_points)

    return resistance_df

resistance_df = find_touch_points(btc_usdt_df, resistance_df)

In [10]:
# Creates 3rd touch boolean for calculating returns later. 
resistance_df['3rd_touch_bool'] = resistance_df['touch_point_3'].notna()
# Every line has minimum of 2 touchpoints already at this point.
resistance_df['num_touch_points'] = resistance_df['num_touch_points'] + 2 
resistance_df['num_touch_points'] = resistance_df['num_touch_points'].astype(int)

In [11]:
resistance_df

,initial_point,end_of_line,touch_2,touch_2_price,line_price,line_length,num_touch_points,touch_point_3,touch_point_3_price,touch_point_4,touch_point_4_price,touch_point_5,touch_point_5_price,touch_point_6,touch_point_6_price,touch_point_7,touch_point_7_price,3rd_touch_bool
0,2020-12-17 06:25:00,2020-12-17 07:20:00,2020-12-17 07:15:00,22288.760,22270.064,11,2,None,None,None,None,None,None,None,None,None,None,False
1,2020-12-17 09:25:00,2020-12-17 13:05:00,2020-12-17 12:55:00,23169.594,23162.428,44,2,None,None,None,None,None,None,None,None,None,None,False
2,2020-12-17 13:35:00,2020-12-17 16:05:00,2020-12-17 15:50:00,23270.470,23326.852,30,2,None,None,None,None,None,None,None,None,None,None,False
3,2020-12-17 16:30:00,2020-12-17 18:15:00,2020-12-17 17:20:00,23587.012,23642.482,21,2,None,None,None,None,None,None,None,None,None,None,False
4,2020-12-17 20:50:00,2020-12-17 22:00:00,2020-12-17 21:40:00,22696.168,22721.138,14,2,None,None,None,None,None,None,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,2021-05-04 18:45:00,2021-05-04 20:50:00,2021-05-04 19:55:00,54552.626,54619.570,25,3,2021-05-04 20:45:00,54560.268,None,None,None,None,None,None,None,None,True
747,2021-05-04 22:55:00,2021-05-05 00:30:00,2021-05-05 00:25:00,54050.680,54103.486,19,2,None,None,None,None,None,None,None,None,None,None,False
748,2021-05-05 01:10:00,2021-05-05 02:10:00,2021-05-05 02:05:00,54928.126,55003.754,12,2,None,None,None,None,None,None,None,None,None,None,False
749,2021-05-05 02:15:00,2021-05-05 06:15:00,2021-05-05 04:30:00,55280.944,55331.866,48,2,None,None,None,None,None,None,None,None,None,None,False


In [10]:
########### calculate_returns_after_third_touch ###########

#--> Calculates the returns after 3rd touch point of a resistance line. Only processes resistance lines that have 3rd touch point.
#--> Uses the closing price at the third touch point as the entry point for the trade.
#--> The data is sliced from the 3rd touch point to the end of a period desired period (3*line_length).
#--> Returns calculated as 'entry_price' - closing candles prices.
#--> The return data is stored in a dictionary indexed by the resistance line ID. 
#--> Good for determining a trading strategy or backtesting/optimizing for SL/TP/etc. (Use bulk data rather than this sample Kraken data)

########### calculate_returns_after_third_touch ###########

In [12]:
def calculate_returns_after_third_touch(df, resistance_df):
    returns_dict = {}

    for idx, row in resistance_df.iterrows():
        if not row['3rd_touch_bool']:
            continue

        line_length = row['line_length']
        third_point = row['touch_point_3']
        
        # Define the entry price as the closing price at the third touch point. (May be a bit below or above the 'line_price' in earlier functions.)
        entry_price = df.loc[df['time'] == third_point, 'close'].values[0]

        # Define the start and end indices for gathering returns data (3x length of line).
        start_idx = df.loc[df['time'] == third_point].index[0]
        end_idx = min(start_idx + 3 * line_length, len(df) - 1)
        # Slice df from the third touch point to the end of the period.
        slice_df = df.iloc[start_idx+1:end_idx+1]
        # Calculate returns based on the entry price and store them in the dictionary.
        returns = slice_df['close'].apply(lambda x: x - entry_price)
        returns_dict[idx] = returns.tolist()

    return returns_dict

returns_dict = calculate_returns_after_third_touch(btc_usdt_df, resistance_df)

In [12]:
########### calculate_trading_results ###########

#--> Calculates the trading results based on returns calculated.
#--> Uses the closing price as the 3rd touch point as the entry point for the trade.
#--> Length of trade is up to 3x length of resistance line.
#--> Defines the stop loss level as 1.3*atr below the entry, and the take profit level as 3*atr above the entry.
#--> Iterates through the sliced data to determine whether the stop loss or take profit was hit first, or neither was hit during the period.
#--> Adds columns to the resistance line DataFrame to store the entry price, stop loss, take profit, the result, timestamp of target hit

########### calculate_trading_results ###########

In [13]:
def calculate_trading_results(df, resistance_df):
    for idx, row in resistance_df.iterrows():
        if not row['3rd_touch_bool']:
            continue

        third_point = row['touch_point_3']
        # Define the entry price as the closing price at the third touch point. 
        entry_price = df.loc[df['time'] == third_point, 'close'].values[0]
        # Define the start time (3rd touch point) and end indices for gathering returns data.
        start_idx = df.loc[df['time'] == third_point].index[0]
        end_idx = len(df)-1
        # Slice the DataFrame from the third touch point to the end of the period.
        slice_df = df.iloc[start_idx+1:end_idx+1]

        result = 'None'
        target_hit_time = 'None'

        # Iterate through the sliced DataFrame to check whether the stop loss or take profit was hit.
        for _, candle in slice_df.iterrows():
            atr = candle['atr']
            sl_level = entry_price - 1.3 * atr
            tp_level = entry_price + 3 * atr

            if candle['high'] >= tp_level:
                result = 'Take Profit'
                target_hit_time = candle['time']
                break
            elif candle['low'] <= sl_level:
                result = 'Stop Loss'
                target_hit_time = candle['time']
                break

        if result == 'None':
            result = 'No Target Hit'

        resistance_df.at[idx, 'entry_price'] = entry_price
        resistance_df.at[idx, 'sl'] = sl_level
        resistance_df.at[idx, 'tp'] = tp_level
        resistance_df.at[idx, 'result'] = result
        resistance_df.at[idx, 'target_hit_time'] = target_hit_time

calculate_trading_results(btc_usdt_df, resistance_df)

In [14]:
resistance_df['touch_point_3'] = pd.to_datetime(resistance_df['touch_point_3'])
resistance_df['target_hit_time'] = pd.to_datetime(resistance_df['target_hit_time'])
# Calculates time in trade using the interval specified earlier. Returns # of candles.
resistance_df['time_in_trade'] = (resistance_df['target_hit_time'] - resistance_df['touch_point_3']).dt.total_seconds() / (time_interval_in_minutes * 60)
resistance_df['time_in_trade'] = resistance_df['time_in_trade'].astype('Int64')
# Calculates net_pl if sl or tp was hit.
def calculate_net_pl(row):
    if row['result'] == 'Take Profit':
        return row['tp'] - row['entry_price']
    elif row['result'] == 'Stop Loss':
        return row['sl'] - row['entry_price']
    else:
        return np.nan
resistance_df['net_pl'] = resistance_df.apply(calculate_net_pl, axis=1)

In [ ]:
# Sorting df: (Automatically takes care of N touch points)

# Define the manually ordered columns before and after touch points
manual_cols_before = ['initial_point', 'end_of_line', 'line_price', 'line_length', 'touch_2', 'touch_2_price']
manual_cols_after = ['num_touch_points', '3rd_touch_bool', 'entry_price', 'sl', 'tp', 'result', 'target_hit_time', 'time_in_trade', 'net_pl']
# Extract all columns starting with 'touch_point_' and sort them
touch_cols = sorted([col for col in resistance_df.columns if col.startswith('touch_point_') and col not in manual_cols_before and col not in manual_cols_after])
# Combine all lists while maintaining the desired order
final_cols = manual_cols_before + touch_cols + manual_cols_after
# Reorganize the DataFrame using the new column order
resistance_df = resistance_df[final_cols]
resistance_df

In [16]:
trades_df = resistance_df[resistance_df['entry_price'].notnull()]
trades_df

,initial_point,end_of_line,line_price,line_length,touch_2,touch_2_price,touch_point_3,touch_point_3_price,touch_point_4,touch_point_4_price,touch_point_5,touch_point_5_price,touch_point_6,touch_point_6_price,touch_point_7,touch_point_7_price,num_touch_points,3rd_touch_bool,entry_price,sl,tp,result,target_hit_time,time_in_trade,net_pl
5,2020-12-17 22:45:00,2020-12-18 02:15:00,23044.328,42,2020-12-18 01:15:00,22996.666,2020-12-18 02:05:00,23065.744,None,None,None,None,None,None,None,None,3,True,23064.56,22945.870,23338.46,Stop Loss,2020-12-18 03:00:00,11,-118.690
8,2020-12-18 17:45:00,2020-12-18 21:00:00,22800.496,39,2020-12-18 18:45:00,22774.192,2020-12-18 20:00:00,22776.656,None,None,None,None,None,None,None,None,3,True,22768.60,22705.407,22914.43,Stop Loss,2020-12-18 20:20:00,4,-63.193
10,2020-12-19 00:10:00,2020-12-19 02:30:00,23080.370,28,2020-12-19 01:05:00,23058.806,2020-12-19 01:55:00,23056.784,None,None,None,None,None,None,None,None,3,True,23046.76,22980.746,23199.10,Take Profit,2020-12-19 02:40:00,9,152.340
12,2020-12-19 10:05:00,2020-12-19 12:35:00,23060.472,30,2020-12-19 11:00:00,23058.158,2020-12-19 12:30:00,23056.586,None,None,None,None,None,None,None,None,3,True,23056.31,22961.436,23275.25,Take Profit,2020-12-19 14:00:00,18,218.940
18,2020-12-20 11:45:00,2020-12-20 15:25:00,23616.618,44,2020-12-20 14:15:00,23596.264,2020-12-20 15:05:00,23599.112,None,None,None,None,None,None,None,None,3,True,23583.40,23483.989,23812.81,Take Profit,2020-12-20 15:45:00,8,229.410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,2021-05-01 07:20:00,2021-05-01 15:10:00,57956.592,94,2021-05-01 08:10:00,57904.210,2021-05-01 12:05:00,58019.094,2021-05-01 13:10:00,57901.576,None,None,None,None,None,None,4,True,58003.15,57829.834,58403.11,Stop Loss,2021-05-01 12:20:00,3,-173.316
734,2021-05-02 14:00:00,2021-05-02 20:55:00,56917.754,83,2021-05-02 14:50:00,56932.400,2021-05-02 18:20:00,56899.99,2021-05-02 19:50:00,56927.418,2021-05-02 20:50:00,56913.12,None,None,None,None,5,True,56899.99,56768.638,57203.11,Stop Loss,2021-05-02 18:30:00,2,-131.352
736,2021-05-03 01:40:00,2021-05-03 07:25:00,58192.658,69,2021-05-03 02:30:00,58161.806,2021-05-03 03:20:00,58176.738,None,None,None,None,None,None,None,None,3,True,58023.46,57850.391,58422.85,Stop Loss,2021-05-03 03:50:00,6,-173.069
745,2021-05-04 15:30:00,2021-05-04 18:40:00,54413.864,38,2021-05-04 17:45:00,54268.894,2021-05-04 18:35:00,54350.388,None,None,None,None,None,None,None,None,3,True,54327.48,54054.714,54956.94,Stop Loss,2021-05-04 20:25:00,22,-272.766


In [21]:
# Returns plot
fig = go.Figure()

# Creates a trace for each item in the returns_dict and adds them to the second figure.
# Each item in returns_dict represents the return series from a particular resistance line, starting from the third touch point.
for idx, (_, returns) in enumerate(returns_dict.items()):
    if idx >= 100:
        break
    scatter_trace = go.Scatter(y=returns, mode='lines', name=f'res_line_{idx + 1}', line=dict(color='#FFD700'))
    fig.add_trace(scatter_trace)

fig.add_hline(y=0, line=dict(color='blue', width=2, dash='dash'))
fig.update_layout(title="Returns from 3rd Touch Point, Long Entry is Close of 3rd Touch Point", xaxis_title="# Candles after 3rd Touch", yaxis_title="Returns", template="plotly_dark")
fig.show()

In [22]:
# Slices the 250,000+ rows of 'btc_usdt_df' so we can plot a more managable chunk.
plt_btc_usdt_df = btc_usdt_df.head(4000)

# Slices resistance_df to match 'plt_btc_usdt_df' length.
last_timestamp = plt_btc_usdt_df['time'].iloc[-1]
mask = resistance_df['end_of_line'] <= last_timestamp
plt_resistance_df = resistance_df[mask]

In [23]:
# Candlestick plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plots candlestick graph with atr, horizontal resistance line, all touch points and target hit point.
candlestick_trace = go.Candlestick(x=plt_btc_usdt_df['time'], open=plt_btc_usdt_df['open'], high=plt_btc_usdt_df['high'], low=plt_btc_usdt_df['low'], close=plt_btc_usdt_df['close'], name='Candlesticks')
atr_trace = go.Scatter(x=plt_btc_usdt_df['time'], y=plt_btc_usdt_df['atr'], mode='lines', name='ATR', line=dict(color='purple'), opacity=0.6)
first_touch_trace = go.Scatter(x=plt_resistance_df['initial_point'], y=plt_resistance_df['line_price'], mode='markers', name='First Touch Point', marker=dict(color='white', size=6))
second_touch_trace = go.Scatter(x=plt_resistance_df['touch_2'], y=plt_resistance_df['touch_2_price'], mode='markers', name='Second Touch Point', marker=dict(color='white', size=6))
target_trace = go.Scatter(x=plt_resistance_df['target_hit_time'], y=plt_resistance_df['net_pl'] + plt_resistance_df['entry_price'], mode='markers', name='Target Hit', marker=dict(color='#FFD700', size=7))

fig.add_trace(candlestick_trace, secondary_y=False)
fig.add_trace(first_touch_trace, secondary_y=False)
fig.add_trace(second_touch_trace, secondary_y=False)

# adding multiple touch point traces to the figure starting with 'touch_point_3'.
for i in range(3, len(plt_resistance_df.columns)):
    touch_point_column = 'touch_point_' + str(i)
    touch_price_column = 'touch_point_' + str(i) + '_price'
    # If a column name exists in the df in the format 'touch_point_i', a trace is created for the touch points in that column and added to graph.
    if touch_point_column in plt_resistance_df.columns:
        touch_point_trace = go.Scatter(x=plt_resistance_df[touch_point_column], y=plt_resistance_df[touch_price_column], mode='markers', name=f'Touch Point {i}', marker=dict(color='#B200ED', size=7))
        fig.add_trace(touch_point_trace, secondary_y=False)

fig.add_trace(target_trace, secondary_y=False)
fig.add_trace(atr_trace, secondary_y=True)

fig.update_layout(title='BTC/USDT', xaxis_title='Time', template="plotly_dark")
fig.update_yaxes(title_text='Price', secondary_y=False, tickformat="$,.2f", autorange=True, fixedrange=False)
fig.update_yaxes(title_text='ATR', secondary_y=True)

# Adds the horizontal resistance lines to graph. Iterates through each row of the resistance_df.
for _, row in plt_resistance_df.iterrows():
    start_time = row['initial_point']
    end_time = row['end_of_line']
    price = row['line_price']
    
    fig.add_shape(type="line", x0=start_time, y0=price, x1=end_time, y1=price, line=dict(color='#39FF14', width=2), xref='x', yref='y')

fig.show()